In [1]:
import selectors
import socket

In [10]:

sel = selectors.DefaultSelector()

def accept(sock, mask):
    conn, addr = sock.accept()  # Should be ready
    print('accepted', conn, 'from', addr)
    conn.setblocking(False)
    sel.register(conn, selectors.EVENT_READ, read)

def read(conn, mask):
    data = conn.recv(1000)  # Should be ready
    if data:
        print('echoing', repr(data), 'to', conn)
        conn.send(data)  # Hope it won't block
    else:
        print('closing', conn)
        sel.unregister(conn)
        conn.close()

serv = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
serv.bind(('localhost', 27002))
serv.listen(100)
serv.setblocking(False)
sel.register(serv, selectors.EVENT_READ, accept)

cli = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
cli.connect(('localhost', 27002))
cli.send(b'test')

# while True:
events = sel.select()
for key, mask in events:
    callback = key.data
    callback(key.fileobj, mask)
    
serv.close()
cli.close()

OSError: [Errno 48] Address already in use